In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# import glob
# import seaborn as sns
# %matplotlib notebook
%matplotlib inline
# from geopy.geocoders import Nominatim
# from geopy.extra.rate_limiter import RateLimiter

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### Import livestock imports and exports

Data for Hogs, Cattle, and Sheep/Lambs

Both interprovincial and international imports and exports are considered

In [8]:
# Cattle 
CattleImportsExports = pd.read_csv('../Datasets/AnimalImportsExports/Cattle/32100139.csv', sep=",", header=0)
# Ojo los valores están en miles!!
CattleImportsExports['VALUE'] = CattleImportsExports['VALUE']*1000
CattleImportsExports = CattleImportsExports.drop('SCALAR_FACTOR', axis=1)
CattleImportsExports = CattleImportsExports[CattleImportsExports['GEO'] == 'Quebec']

CattleImports = CattleImportsExports[CattleImportsExports['Supply and disposition of cattle'].str.contains('imports')==True].groupby('REF_DATE').sum(numeric_only=True)
CattleImports = CattleImports.rename(columns={'VALUE':'Cattle /Imports (heads)'})
CattleExports = CattleImportsExports[CattleImportsExports['Supply and disposition of cattle'].str.contains('exports')==True].groupby('REF_DATE').sum(numeric_only=True)
CattleExports = CattleExports.rename(columns={'VALUE':'Cattle /Exports (heads)'})

# Swine 
SwineImportsExports = pd.read_csv('../Datasets/AnimalImportsExports/Hogs/32100200.csv', sep=",", header=0)
# Ojo los valores están en miles!!
SwineImportsExports['VALUE'] = SwineImportsExports['VALUE']*1000
SwineImportsExports = SwineImportsExports.drop('SCALAR_FACTOR', axis=1)
SwineImportsExports = SwineImportsExports[SwineImportsExports['GEO'] == 'Quebec']

SwineImports = SwineImportsExports[SwineImportsExports['Supply and disposition of hogs'].str.contains('imports')==True].groupby('REF_DATE').sum(numeric_only=True)
SwineImports = SwineImports.rename(columns={'VALUE':'Hogs /Imports (heads)'})
SwineExports = SwineImportsExports[SwineImportsExports['Supply and disposition of hogs'].str.contains('exports')==True].groupby('REF_DATE').sum(numeric_only=True)
SwineExports = SwineExports.rename(columns={'VALUE':'Hogs /Exports (heads)'})

# Sheep and lamb 
SheepImportsExports = pd.read_csv('../Datasets/AnimalImportsExports/Sheep and lambs/32100141.csv', sep=",", header=0)
# Ojo los valores están en miles!!
SheepImportsExports['VALUE'] = SheepImportsExports['VALUE']*1000
SheepImportsExports = SheepImportsExports.drop('SCALAR_FACTOR', axis=1)
SheepImportsExports = SheepImportsExports[SheepImportsExports['GEO'] == 'Quebec']

SheepImports = SheepImportsExports[SheepImportsExports['Supply and disposition of sheep and lambs'].str.contains('imports')==True].groupby('REF_DATE').sum(numeric_only=True)
SheepImports = SheepImports.rename(columns={'VALUE':'Sheep /Imports (heads)'})
SheepExports = SheepImportsExports[SheepImportsExports['Supply and disposition of sheep and lambs'].str.contains('exports')==True].groupby('REF_DATE').sum(numeric_only=True)
SheepExports = SheepExports.rename(columns={'VALUE':'Sheep /Exports (heads)'})


SheepExports

,UOM_ID,SCALAR_ID,Sheep /Exports (heads),SYMBOL,DECIMALS
REF_DATE,,,,,
2000,592,12,31600.0,0.0,4
2001,592,12,29000.0,0.0,4
2002,592,12,64500.0,0.0,4
2003,592,12,67400.0,0.0,4
2004,592,12,57000.0,0.0,4
2005,592,12,57800.0,0.0,4
2006,888,18,79400.0,0.0,6
2007,888,18,67300.0,0.0,6
2008,888,18,101300.0,0.0,6


In [9]:
LivestockImportsExports = pd.DataFrame()

LivestockImportsExports['REF_DATE'] = CattleImports.index.to_list()
LivestockImportsExports = LivestockImportsExports.set_index('REF_DATE')

LivestockImportsExports = LivestockImportsExports.merge(CattleImports['Cattle /Imports (heads)'], left_index=True, right_index=True)
LivestockImportsExports = LivestockImportsExports.merge(CattleExports['Cattle /Exports (heads)'], left_index=True, right_index=True)

LivestockImportsExports = LivestockImportsExports.merge(SwineImports['Hogs /Imports (heads)'], left_index=True, right_index=True)
LivestockImportsExports = LivestockImportsExports.merge(SwineExports['Hogs /Exports (heads)'], left_index=True, right_index=True)

LivestockImportsExports = LivestockImportsExports.merge(SheepImports['Sheep /Imports (heads)'], left_index=True, right_index=True)
LivestockImportsExports = LivestockImportsExports.merge(SheepExports['Sheep /Exports (heads)'], left_index=True, right_index=True)

LivestockImportsExports.index = LivestockImportsExports.index.astype(str)

# We convert everyting to multirow
iterables = [LivestockImportsExports.index.to_list(), LivestockImportsExports.columns.to_list()]
multiindex_LivestockImportsExport = pd.MultiIndex.from_product(iterables, names=["Year", "Animal"])


LivestockImportsExports_Conditioned = pd.DataFrame(index=['Total Quebec'], columns=multiindex_LivestockImportsExport)
for i in LivestockImportsExports.index.to_list():
    for ii in LivestockImportsExports.columns.to_list():
        LivestockImportsExports_Conditioned.loc['Total Quebec',(i,ii)] = LivestockImportsExports.loc[i,ii]

LivestockImportsExports_Conditioned

Year                            2007                                                                                                                                      2008                                                                                                                                      2009                                                                                                                                      2010                                                                                                                                      2011                                                                                                                                      2012                                                                                                                                      2013                                                                                                                    \
Animal       Cattle /Imports (heads) Cattle /Exports (heads) Hogs /Imports (heads) Hogs /Exports (heads) Sheep /Imports (heads) Sheep /Exports (heads) Cattle /Imports (heads) Cattle /Exports (heads) Hogs /Imports (heads) Hogs /Exports (heads) Sheep /Imports (heads) Sheep /Exports (heads) Cattle /Imports (heads) Cattle /Exports (heads) Hogs /Imports (heads) Hogs /Exports (heads) Sheep /Imports (heads) Sheep /Exports (heads) Cattle /Imports (heads) Cattle /Exports (heads) Hogs /Imports (heads) Hogs /Exports (heads) Sheep /Imports (heads) Sheep /Exports (heads) Cattle /Imports (heads) Cattle /Exports (heads) Hogs /Imports (heads) Hogs /Exports (heads) Sheep /Imports (heads) Sheep /Exports (heads) Cattle /Imports (heads) Cattle /Exports (heads) Hogs /Imports (heads) Hogs /Exports (heads) Sheep /Imports (heads) Sheep /Exports (heads) Cattle /Imports (heads) Cattle /Exports (heads) Hogs /Imports (heads) Hogs /Exports (heads) Sheep /Imports (heads) Sheep /Exports (heads)   
Total Quebec                182700.0                131300.0               72700.0               37900.0                18700.0                67300.0                200500.0                170000.0              552000.0               56100.0                29600.0               101300.0                149200.0                121700.0              464900.0               54800.0                15000.0                79600.0                141800.0                124300.0              509400.0               59300.0                10000.0                82800.0                145400.0                116100.0              640200.0               45300.0                10800.0                68200.0                100000.0                120800.0              769500.0               36900.0                13000.0                62400.0                102400.0                141300.0              841300.0               26500.0                12600.0                62900.0   

Year                            2014                                                                                                                                      2015                                                                                                                                      2016                                                                                                                                      2017                                                                                                                                      2018                                                                                                                                      2019                                                                                                                                      2020                                                                                                                    \
Animal       Cattle /Imports (heads) Cattle /Exports (heads) Hogs /Impo

In [10]:
LivestockImportsExports

,Cattle /Imports (heads),Cattle /Exports (heads),Hogs /Imports (heads),Hogs /Exports (heads),Sheep /Imports (heads),Sheep /Exports (heads)
REF_DATE,,,,,,
2007,182700.0,131300.0,72700.0,37900.0,18700.0,67300.0
2008,200500.0,170000.0,552000.0,56100.0,29600.0,101300.0
2009,149200.0,121700.0,464900.0,54800.0,15000.0,79600.0
2010,141800.0,124300.0,509400.0,59300.0,10000.0,82800.0
2011,145400.0,116100.0,640200.0,45300.0,10800.0,68200.0
2012,100000.0,120800.0,769500.0,36900.0,13000.0,62400.0
2013,102400.0,141300.0,841300.0,26500.0,12600.0,62900.0
2014,78600.0,126100.0,660100.0,24600.0,10200.0,64700.0
2015,68100.0,119600.0,701600.0,24400.0,8300.0,66200.0


### Animal Composition Datasets

In [11]:
AnimalComposition = pd.read_csv('../Datasets/AgriProductsProduction/Meat/Animal compositions/AnimalCompositions.csv', sep=",", header=0).set_index('Animal')
AnimalComposition.columns
AnimalComposition

,P (%live_wt),Ref
Animal,,
Cattle,0.740,Standardised calculation methods for\nanimal m...
Hogs,0.532,Standardised calculation methods for\nanimal m...
Calves,0.800,Standardised calculation methods for\nanimal m...
Sheep,0.520,Standardised calculation methods for\nanimal m...
Goat,0.630,Standardised calculation methods for\nanimal m...
Rabbit,0.600,Standardised calculation methods for\nanimal m...
Chicken,0.500,Standardised calculation methods for\nanimal m...
Turkey,0.510,Standardised calculation methods for\nanimal m...


### Carcass weight datasets


In [12]:
LivetoCarcassWeight_ratio = pd.read_csv('../Datasets/AgriProductsProduction/Meat/LiveCarcassWeightRatio_Conditioned.csv', sep=",", header=0).set_index('Animal')
LivetoCarcassWeight_ratio

,Carcass_Live_ratio (%)
Animal,
Cattle,49
Hogs,80
Calves,55
Sheep,46
Goat,50
Rabbit,53
Chicken,72
Turkey,77


In [13]:
CarcassWeight = pd.read_csv('../Datasets/AgriProductsProduction/Meat/WeightCarcasses_Final.csv', sep=",", header=0).set_index('Animals')
CarcassWeight['Average'] = CarcassWeight.mean(axis=1)
CarcassWeight

,2016,2017,2018,2019,2020,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2021,Average
Animals,,,,,,,,,,,,,,,,,,,,,,
Cattle,404.628701,401.045536,398.970340,408.395853,403.375141,404.628701,404.628701,404.628701,404.628701,404.628701,404.628701,404.628701,404.628701,404.628701,404.628701,404.628701,404.628701,404.628701,404.628701,404.628701,403.283114,404.244247
Hogs,105.766719,104.196644,105.240872,106.667994,109.382161,105.766719,105.766719,105.766719,105.766719,105.766719,105.766719,105.766719,105.766719,105.766719,105.766719,105.766719,105.766719,105.766719,105.766719,105.766719,106.250878,105.905050
Calves,173.302053,173.008302,170.627274,172.843052,180.874546,173.302053,173.302053,173.302053,173.302053,173.302053,173.302053,173.302053,173.302053,173.302053,173.302053,173.302053,173.302053,173.302053,173.302053,173.302053,174.131045,173.538908
Sheep,19.700000,19.700000,19.700000,19.700000,19.700000,19.700000,19.700000,19.700000,19.700000,19.700000,19.700000,19.700000,19.700000,19.700000,19.700000,19.700000,19.700000,19.700000,19.700000,19.700000,19.700000,19.700000
Goat,14.190000,14.190000,14.190000,14.190000,14.190000,14.190000,14.190000,14.190000,14.190000,14.190000,14.190000,14.190000,14.190000,14.190000,14.190000,14.190000,14.190000,14.190000,14.190000,14.190000,14.190000,14.190000
Rabbit,1.250000,1.250000,1.250000,1.250000,1.250000,1.250000,1.250000,1.250000,1.250000,1.250000,1.250000,1.250000,1.250000,1.250000,1.250000,1.250000,1.250000,1.250000,1.250000,1.250000,1.250000,1.250000
Chicken,1.753357,1.730572,1.766939,1.784729,1.782761,1.753357,1.753357,1.753357,1.753357,1.753357,1.753357,1.753357,1.753357,1.753357,1.753357,1.753357,1.753357,1.753357,1.753357,1.753357,1.763672,1.756304
Turkey,8.358019,8.279067,8.261124,8.294842,8.501426,8.358019,8.358019,8.358019,8.358019,8.358019,8.358019,8.358019,8.358019,8.358019,8.358019,8.358019,8.358019,8.358019,8.358019,8.358019,8.338896,8.352555


In [14]:
LivestockImportsExports_P = LivestockImportsExports_Conditioned.copy()

LivestockImportsExports_index = LivestockImportsExports_Conditioned.index.to_list()
LivestockImportsExports_columns = LivestockImportsExports_Conditioned.columns.to_list()

for i in LivestockImportsExports_index:
    for ii in LivestockImportsExports_columns:
        LivestockImportsExports_P.loc[i,(ii[0],ii[1].split(' (heads)')[0]  +' (kg live)')] = (LivestockImportsExports_P.loc[i,ii]*CarcassWeight.loc[ii[1].split(' /')[0],'Average']/
                                                              (LivetoCarcassWeight_ratio.loc[ii[1].split(' /')[0],'Carcass_Live_ratio (%)']/100))
        
        LivestockImportsExports_P.loc[i,(ii[0],ii[1].split(' (heads)')[0]  +' (kg P live)')] = (LivestockImportsExports_P.loc[i,(ii[0],ii[1].split(' (heads)')[0]  +' (kg live)')]*
                                                                                                AnimalComposition.loc[ii[1].split(' /')[0],'P (%live_wt)']/100)

LivestockImportsExports_P.to_csv('../Results/AnimalImportsExports/LivestockImportsExports_PRawQuebec.csv')
LivestockImportsExports_P

/tmp/ipykernel_28636/623394220.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  LivestockImportsExports_P.loc[i,(ii[0],ii[1].split(' (heads)')[0]  +' (kg P live)')] = (LivestockImportsExports_P.loc[i,(ii[0],ii[1].split(' (heads)')[0]  +' (kg live)')]*
/tmp/ipykernel_28636/623394220.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  LivestockImportsExports_P.loc[i,(ii[0],ii[1].split(' (heads)')[0]  +' (kg live)')] = (LivestockImportsExports_P.loc[i,ii]*CarcassWeight.loc[ii[1].split(' /')[0],'Average']/


Year                            2007                                                                                                                                      2008                                                                                                                                      2009                                                                                                                                      2010                                                                                                                                      2011                                                                                                                                      2012                                                                                                                                      2013                                                                                                                    \
Animal       Cattle /Imports (heads) Cattle /Exports (heads) Hogs /Imports (heads) Hogs /Exports (heads) Sheep /Imports (heads) Sheep /Exports (heads) Cattle /Imports (heads) Cattle /Exports (heads) Hogs /Imports (heads) Hogs /Exports (heads) Sheep /Imports (heads) Sheep /Exports (heads) Cattle /Imports (heads) Cattle /Exports (heads) Hogs /Imports (heads) Hogs /Exports (heads) Sheep /Imports (heads) Sheep /Exports (heads) Cattle /Imports (heads) Cattle /Exports (heads) Hogs /Imports (heads) Hogs /Exports (heads) Sheep /Imports (heads) Sheep /Exports (heads) Cattle /Imports (heads) Cattle /Exports (heads) Hogs /Imports (heads) Hogs /Exports (heads) Sheep /Imports (heads) Sheep /Exports (heads) Cattle /Imports (heads) Cattle /Exports (heads) Hogs /Imports (heads) Hogs /Exports (heads) Sheep /Imports (heads) Sheep /Exports (heads) Cattle /Imports (heads) Cattle /Exports (heads) Hogs /Imports (heads) Hogs /Exports (heads) Sheep /Imports (heads) Sheep /Exports (heads)   
Total Quebec                182700.0                131300.0               72700.0               37900.0                18700.0                67300.0                200500.0                170000.0              552000.0               56100.0                29600.0               101300.0                149200.0                121700.0              464900.0               54800.0                15000.0                79600.0                141800.0                124300.0              509400.0               59300.0                10000.0                82800.0                145400.0                116100.0              640200.0               45300.0                10800.0                68200.0                100000.0                120800.0              769500.0               36900.0                13000.0                62400.0                102400.0                141300.0              841300.0               26500.0                12600.0                62900.0   

Year                            2014                                                                                                                                      2015                                                                                                                                      2016                                                                                                                                      2017                                                                                                                                      2018                                                                                                                                      2019                                                                                                                                      2020                                                                                                                    \
Animal       Cattle /Imports (heads) Cattle /Exports (heads) Hogs /Impo

In [15]:
LivestockImportsExports_PClean = LivestockImportsExports_P[LivestockImportsExports_P.filter(regex='kg P').columns].copy()
LivestockImportsExports_PClean.to_csv('../Results/AnimalImportsExports/LivestockImportsExports_PCleanQuebec.csv')

LivestockImportsExports_PClean

Year                                2007                                                                                                                                                              2008                                                                                                                                                              2009                                                                                                                                                              2010                                                                                                                                                              2011                                                                                                                                                              2012                                                                                                                                        \
Animal       Cattle /Imports (kg P live) Cattle /Exports (kg P live) Hogs /Imports (kg P live) Hogs /Exports (kg P live) Sheep /Imports (kg P live) Sheep /Exports (kg P live) Cattle /Imports (kg P live) Cattle /Exports (kg P live) Hogs /Imports (kg P live) Hogs /Exports (kg P live) Sheep /Imports (kg P live) Sheep /Exports (kg P live) Cattle /Imports (kg P live) Cattle /Exports (kg P live) Hogs /Imports (kg P live) Hogs /Exports (kg P live) Sheep /Imports (kg P live) Sheep /Exports (kg P live) Cattle /Imports (kg P live) Cattle /Exports (kg P live) Hogs /Imports (kg P live) Hogs /Exports (kg P live) Sheep /Imports (kg P live) Sheep /Exports (kg P live) Cattle /Imports (kg P live) Cattle /Exports (kg P live) Hogs /Imports (kg P live) Hogs /Exports (kg P live) Sheep /Imports (kg P live) Sheep /Exports (kg P live) Cattle /Imports (kg P live) Cattle /Exports (kg P live) Hogs /Imports (kg P live) Hogs /Exports (kg P live) Sheep /Imports (kg P live) Sheep /Exports (kg P live)   
Total Quebec                1.115368e+06               801575.092987              51200.326177              26691.779396                4164.408696               14987.417391                1.224035e+06                1.037835e+06             388756.259281              39509.467655                6591.791304               22559.069565                910853.03788               742967.927011              327414.46547              38593.918494                3340.434783               17726.573913               865676.680773               758840.701129             358754.417532              41763.127129                2226.956522                    18439.2               887654.368015               708780.413525             450872.748536               31903.36693                2405.113043               15187.843478               610491.312252               737473.505201             541934.676661              25987.510811                2895.043478               13896.208696   

Year                                2013                                                                                                                                                              2014                                                                                                                                                              2015                                                                                                                                                              2016                                                                                                                                                              2017                                                                                                                                                              2018                                                                                                                                        \
Animal       Cattle /Imports (kg P live) Cattle

In [16]:
LivestockImportsExports_P2001to2021_aux = pd.read_csv('../Results/AnimalImportsExports/LivestockImportsExports_PCleanQuebec.csv', sep=",",header=None, index_col=[0])

for name, values in LivestockImportsExports_P2001to2021_aux.iteritems():
    # print('{name}: {value}'.format(name=name, value=values[1]))
    LivestockImportsExports_P2001to2021_aux.loc['Name', name] = LivestockImportsExports_P2001to2021_aux.loc['Animal', name]+' '+LivestockImportsExports_P2001to2021_aux.loc['Year', name]
    
LivestockImportsExports_P2001to2021_aux = LivestockImportsExports_P2001to2021_aux.rename(columns=dict(zip(LivestockImportsExports_P2001to2021_aux.columns.to_list(),LivestockImportsExports_P2001to2021_aux.loc['Name',:].to_list())))

LivestockImportsExports_P2001to2021_aux = LivestockImportsExports_P2001to2021_aux.drop(['Year','Animal', 'Name'], axis=0)
LivestockImportsExports_P2001to2021_aux.index.names = ['CD']
LivestockImportsExports_P2001to2021_aux.index = LivestockImportsExports_P2001to2021_aux.index

LivestockImportsExports_P2001to2021_aux

/tmp/ipykernel_28636/1709807533.py:3: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, values in LivestockImportsExports_P2001to2021_aux.iteritems():


,Cattle /Imports (kg P live) 2007,Cattle /Exports (kg P live) 2007,Hogs /Imports (kg P live) 2007,Hogs /Exports (kg P live) 2007,Sheep /Imports (kg P live) 2007,Sheep /Exports (kg P live) 2007,Cattle /Imports (kg P live) 2008,Cattle /Exports (kg P live) 2008,Hogs /Imports (kg P live) 2008,Hogs /Exports (kg P live) 2008,Sheep /Imports (kg P live) 2008,Sheep /Exports (kg P live) 2008,Cattle /Imports (kg P live) 2009,Cattle /Exports (kg P live) 2009,Hogs /Imports (kg P live) 2009,Hogs /Exports (kg P live) 2009,Sheep /Imports (kg P live) 2009,Sheep /Exports (kg P live) 2009,Cattle /Imports (kg P live) 2010,Cattle /Exports (kg P live) 2010,Hogs /Imports (kg P live) 2010,Hogs /Exports (kg P live) 2010,Sheep /Imports (kg P live) 2010,Sheep /Exports (kg P live) 2010,Cattle /Imports (kg P live) 2011,Cattle /Exports (kg P live) 2011,Hogs /Imports (kg P live) 2011,Hogs /Exports (kg P live) 2011,Sheep /Imports (kg P live) 2011,Sheep /Exports (kg P live) 2011,Cattle /Imports (kg P live) 2012,Cattle /Exports (kg P live) 2012,Hogs /Imports (kg P live) 2012,Hogs /Exports (kg P live) 2012,Sheep /Imports (kg P live) 2012,Sheep /Exports (kg P live) 2012,Cattle /Imports (kg P live) 2013,Cattle /Exports (kg P live) 2013,Hogs /Imports (kg P live) 2013,Hogs /Exports (kg P live) 2013,Sheep /Imports (kg P live) 2013,Sheep /Exports (kg P live) 2013,Cattle /Imports (kg P live) 2014,Cattle /Exports (kg P live) 2014,Hogs /Imports (kg P live) 2014,Hogs /Exports (kg P live) 2014,Sheep /Imports (kg P live) 2014,Sheep /Exports (kg P live) 2014,Cattle /Imports (kg P live) 2015,Cattle /Exports (kg P live) 2015,Hogs /Imports (kg P live) 2015,Hogs /Exports (kg P live) 2015,Sheep /Imports (kg P live) 2015,Sheep /Exports (kg P live) 2015,Cattle /Imports (kg P live) 2016,Cattle /Exports (kg P live) 2016,Hogs /Imports (kg P live) 2016,Hogs /Exports (kg P live) 2016,Sheep /Imports (kg P live) 2016,Sheep /Exports (kg P live) 2016,Cattle /Imports (kg P live) 2017,Cattle /Exports (kg P live) 2017,Hogs /Imports (kg P live) 2017,Hogs /Exports (kg P live) 2017,Sheep /Imports (kg P live) 2017,Sheep /Exports (kg P live) 2017,Cattle /Imports (kg P live) 2018,Cattle /Exports (kg P live) 2018,Hogs /Imports (kg P live) 2018,Hogs /Exports (kg P live) 2018,Sheep /Imports (kg P live) 2018,Sheep /Exports (kg P live) 2018,Cattle /Imports (kg P live) 2019,Cattle /Exports (kg P live) 2019,Hogs /Imports (kg P live) 2019,Hogs /Exports (kg P live) 2019,Sheep /Imports (kg P live) 2019,Sheep /Exports (kg P live) 2019,Cattle /Imports (kg P live) 2020,Cattle /Exports (kg P live) 2020,Hogs /Imports (kg P live) 2020,Hogs /Exports (kg P live) 2020,Sheep /Imports (kg P live) 2020,Sheep /Exports (kg P live) 2020,Cattle /Imports (kg P live) 2021,Cattle /Exports (kg P live) 2021,Hogs /Imports (kg P live) 2021,Hogs /Exports (kg P live) 2021,Sheep /Imports (kg P live) 2021,Sheep /Exports (kg P live) 2021,Cattle /Imports (kg P live) 2022,Cattle /Exports (kg P live) 2022,Hogs /Imports (kg P live) 2022,Hogs /Exports (kg P live) 2022,Sheep /Imports (kg P live) 2022,Sheep /Exports (kg P live) 2022
CD,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Total Quebec,1115367.627484596,801575.0929870141,51200.326177038354,26691.77939628272,4164.408695652173,14987.417391304345,1224035.0810654708,1037835.2308285789,388756.25928095146,39509.46765518366,6591.791304347825,22559.06956521739,910853.0378801409,742967.927010812,327414.46547049703,38593.9184938336,3340.434782608695,17726.573913043478,865676.6807734851,758840.7011293668,358754.41753209545,41763.12712927613,2226.95652173913,18439.199999999993,887654.3680145609,708780.4135246942,450872.74853562535,31903.36693012156,2405.1130434782604,15187.843478260867,610491.3122521052,737473.505200543,541934.6766606742,25987.51081062882,2895.043478260869,13896.208695652173,625143.1037461556,862624.2242122246,592501.1611106242,18663.117519828287,2805.965217391303,14007.556521739129,479846.17143015465,769829.5447499048,464887.6933

In [17]:
years_complete =  np.arange(2007,2021+1,1)
years_complete.sort()

# EggProductionQuebec_CD2001to2021 = pd.DataFrame(np.nan, index=range(len(EggProductionQuebec_CD.CD)*len(years_complete)), columns=['DGUID'])
DGUID_list = []
CD_list = []
REF_DATE_list = []
Animals_list = []
for CD in LivestockImportsExports_P2001to2021_aux.index:
    for year in years_complete:
        for animal in ['Hogs','Sheep','Cattle']:
            DGUID_list.append(str(year)+'A0003'+str(CD))
            CD_list.append(CD)
            REF_DATE_list.append(year)
            Animals_list.append(animal)
        
LivestockImportsExports_P2001to2021 = pd.DataFrame({'DGUID':DGUID_list,
                                            'CD':CD_list,
                                            'REF_DATE':REF_DATE_list,
                                            'Animal':Animals_list})

LivestockImportsExports_P2001to2021 = LivestockImportsExports_P2001to2021.sort_values(by=['CD', 'REF_DATE'])


for index, row in LivestockImportsExports_P2001to2021.iterrows():
    year = str(LivestockImportsExports_P2001to2021.loc[index,'REF_DATE'])
    CenDiv = LivestockImportsExports_P2001to2021.loc[index,'CD']
    Animal = LivestockImportsExports_P2001to2021.loc[index,'Animal']
    
    label1 = Animal + ' /Imports (kg P live) '+ year
    label2 = Animal + ' /Exports (kg P live) '+ year
    
    LivestockImportsExports_P2001to2021.loc[index,'Imports (kg P live)'] = LivestockImportsExports_P2001to2021_aux.loc[CenDiv,label1]
    LivestockImportsExports_P2001to2021.loc[index,'Exports (kg P live)'] = LivestockImportsExports_P2001to2021_aux.loc[CenDiv,label2]

LivestockImportsExports_P2001to2021 = LivestockImportsExports_P2001to2021.set_index('DGUID')
LivestockImportsExports_P2001to2021.to_csv('../Results/AnimalImportsExports/LivestockImportsExports_P2001to2021.csv')
LivestockImportsExports_P2001to2021

,CD,REF_DATE,Animal,Imports (kg P live),Exports (kg P live)
DGUID,,,,,
2007A0003Total Quebec,Total Quebec,2007,Hogs,51200.326177038354,26691.77939628272
2007A0003Total Quebec,Total Quebec,2007,Sheep,4164.408695652173,14987.417391304345
2007A0003Total Quebec,Total Quebec,2007,Cattle,1115367.627484596,801575.0929870141
2008A0003Total Quebec,Total Quebec,2008,Hogs,388756.25928095146,39509.46765518366
2008A0003Total Quebec,Total Quebec,2008,Sheep,6591.791304347825,22559.06956521739
2008A0003Total Quebec,Total Quebec,2008,Cattle,1224035.0810654708,1037835.2308285789
2009A0003Total Quebec,Total Quebec,2009,Hogs,327414.46547049703,38593.9184938336
2009A0003Total Quebec,Total Quebec,2009,Sheep,3340.434782608695,17726.573913043478
2009A0003Total Quebec,Total Quebec,2009,Cattle,910853.0378801409,742967.927010812


In [18]:
LivestockImportsExports_PSummary = LivestockImportsExports_PClean[LivestockImportsExports_PClean.filter(regex='2021').columns].copy()
LivestockImportsExports_PSummary.to_csv('../Results/AnimalImportsExports/LivestockImportsExports_PSummary2021Quebec.csv')

LivestockImportsExports_PSummary

Year                                2021                                                                                                                                      
Animal       Cattle /Imports (kg P live) Cattle /Exports (kg P live) Hogs /Imports (kg P live) Hogs /Exports (kg P live) Sheep /Imports (kg P live) Sheep /Exports (kg P live)
Total Quebec               239312.594403               435890.796948             613840.499256             375656.863588                1358.443478               16679.904348